# CME Smart Stream on Google Cloud Platform Tutorials
## Getting CME Binary Data from CME Smart Stream on Google Cloud Platform (GCP)

This workbook demonstrates the ability to quickly use the CME Smart Stream on GCP solution.  Through the examples, we will 
- Authenticate using GCP IAM information
- Configure which CME Smart Stream on GCP Topic containing the Market Data 
- Download a single message from your Cloud Pub/Sub Subscription
- Delete your Cloud Pub/Sub Subscription

The following example references the following webpage to pull the information:

https://www.cmegroup.com/confluence/display/EPICSANDBOX/CME+Smart+Stream+GCP+Topic+Names


Author:  Aaron Walters (Github: @aaronwalters79).  
OS: MacOS

In [1]:
#import packages.  These are outlined in the environment.ymal file as part of this project.
#these can also be directly imported. 

# Google SDK: https://cloud.google.com/sdk/docs/quickstarts
# Google PubSub: https://cloud.google.com/pubsub/docs/reference/libraries

from google.cloud import pubsub_v1
import os
import google.auth



# Authentication using Google IAM

CME Smart Stream uses Google Cloud native Idenity and Accesss Management (IAM) solutions.  Using this approach, customers are able to natively access CME Smart Stream solution without custom SDK's or authentication routines.  All the code in this workboard is native Google Python SDK.  While the Google Pub/Sub examples below are using python, there are native SDK's for other popular languages including Java, C#, Node.js, PHP, and others.

To download those libraries, please see the following location: https://cloud.google.com/pubsub/docs/reference/libraries


When onboarding to CME Smart Stream, you will supply at least one Google IAM Member accounts.  https://cloud.google.com/iam/docs/overview.  When accessing CME Smart Stream Topics, you will use the same IAM account infromation to create your Subscription using navative GCP authenticaion routines within the GCP SDK. 

The following authentication routines below use either a Service Account or User Account.  Google highly recomends using an Service Account with associated authorization json.  This document also contains authentication via User Account in the event you requested CME to use User Account for access.  You only need to use one of these for the example.



## Authentication Routine for Service Account  
This section is for customers using Service Accounts.  You should update the './gcp-auth.json' to reference your local authorization json file downloaded from google.  

Further documentation is located here: https://cloud.google.com/docs/authentication/getting-started    

In [2]:
## Authentication Method Options -- SERVICE ACCOUNT JSON FILE
# This should point to the file location of the json file downloaded from GCP console.  This will load it into your os variables and be automtically leverage when your system interacts with GCP.

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./gcp-auth.json"  #Uncomment if using this method.



## Authentication for User Account

This section is for customers that registered thier GCP User Account (i.e. user@domain.com).  This routine will launch the OS SDK to authenticate you as that user and then set it as your default credentials for the rest of the workflow when interacting with GCP.  

IN OS TERMINAL: 'gcloud auth application-default login'  without quotes.

In [4]:
## Authentication Method User Machine Defaults
#
#Run "gcloud auth login" in command line and login as the user. The code below will do that automatically.  
#It should laucnh a browser to authenticate into GCP that user name and associated permissions will be used in the remaining of the code below

# This code will put out a warning about using end user credentials.
# Reference: https://google-auth.readthedocs.io/en/latest/user-guide.html


credentials, project = google.auth.default()

/anaconda3/envs/python3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# Set Your Smart Stream on GCP Projects and Topics



## Set CME Smart Stream Project 

CME Smart Stream on GCP data is avaliable in two GCP Projects based upon Production and Non-Production (i.e. certification and new release) data.  Customers are granted access to projects through the onboarding process. 

The example below sets the target CME Smart Stream on GCP Project as an OS variable for easy reference.


In [5]:
#This is the project at CME 
os.environ['GOOGLE_CLOUD_PROJECT_CME'] = "cmegroup-marketdata-newrel" #CERT and NEW RELEASE
#os.environ['GOOGLE_CLOUD_PROJECT_CME'] = "cmegroup-marketdata"  #PRODUCTION

## Set CME Smart Stream Topics

CME Smart Stream on GCP follows the traditional data segmentation of the CME Multicast solution. 

Each channel on Multicast is avaliable as a Topic in Google Cloud PubSub.  This workbook will create 1 subscription in the customer's account against 1 Topic from the CME project.  Clearly, customers can script this to create as many subscriptions as needed.  

Please see: https://www.cmegroup.com/confluence/display/EPICSANDBOX/CME+Smart+Stream+GCP+Topic+Names for all the topic names. 

You can also review the notebook included in this git project named Google PubSub Get CME Topics on how to read the names from the website into a local CSV file or use in automated scripts.

In [6]:
# The CME TOPIC that a Subscription will be created against
os.environ['GOOGLE_CLOUD_TOPIC_CME'] = "CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310"  #CERT
#s.environ['GOOGLE_CLOUD_TOPIC_CME'] = "NR.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310" #NEW RELEASE
#os.environ['GOOGLE_CLOUD_TOPIC_CME'] = "CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310" #PRODUCTION 



# Set Customer Configurations

## Set Customer Project & Subscription Name

Smart Stream on GCP solution requires that the customer create a Cloud Pub/Sub Subscription in thier account.  This subscription will automatically collect data from the CME Smart Stream Pub/Sub Topic. Since the Subscriptin is in the customer account we must specify the customer GCP Project and the name of the Subscription they want in the project.  

In the example below, we set the project directly based upon our GCP project name.  We also create a subscription name by prepending 'MY_' to the name of the Topic we are joining.  


In [14]:
#Your Configurations for the project you want to have access; 
#will use the defaults from credentials 
os.environ['GOOGLE_CLOUD_PROJECT'] = "prefab-rampart-794"

#My Subscipription Name -- Take the CME Topic Name and prepend 'MY_' -- Can be any thing the customer wants
os.environ['MY_SUBSCRIPTION_NAME'] = 'MY_'+os.environ['GOOGLE_CLOUD_TOPIC_CME'] #MY SUBSCRIPTION NAME

# Final Configuration

The following is the final configuration for your setup.

In [15]:

print ('Target Project: \t',os.environ['GOOGLE_CLOUD_PROJECT_CME'] )
print ('Target Topic: \t\t', os.environ['GOOGLE_CLOUD_TOPIC_CME'] , '\n' )
print ('My Project: \t\t',os.environ['GOOGLE_CLOUD_PROJECT'])
print ('My Subscriptions: \t',os.environ['MY_SUBSCRIPTION_NAME'] )

Target Project: 	 cmegroup-marketdata-newrel
Target Topic: 		 CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310 

My Project: 		 prefab-rampart-794
My Subscriptions: 	 ALEXDEMO_CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310


# Create Your Subscription to CME Smart Stream Data Topics
We have all the main variables set and can pass them to the Cloud Pub/Sub Python SDK.  The following attempts to create a Subscription (MY_SUBSCRIPTION_NAME) in your specified project (GOOGLE_CLOUD_PROJECT) that points to the CME Topic (GOOGLE_CLOUD_TOPIC_CME) and Project (GOOGLE_CLOUD_PROJECT_CME) of that Topic.

Once created or determined it already exists we will join our python session to the Subscription as 'subscriber'. 

Full documentation on this Pub/Sub example is avaliable: https://googleapis.github.io/google-cloud-python/latest/pubsub/#subscribing


In [19]:
#https://googleapis.github.io/google-cloud-python/latest/pubsub/#subscribing


#Create Topic Name from Config Above
topic_name = 'projects/{cme_project_id}/topics/{cme_topic}'.format( cme_project_id=os.getenv('GOOGLE_CLOUD_PROJECT_CME'), cme_topic=os.getenv('GOOGLE_CLOUD_TOPIC_CME'),  )

#Create Subscription Name from Config Above
subscription_name = 'projects/{my_project_id}/subscriptions/{my_sub}'.format(my_project_id=os.getenv('GOOGLE_CLOUD_PROJECT'),my_sub=os.environ['MY_SUBSCRIPTION_NAME'],  )

#Try To Create a subscription in your Project
subscriber = pubsub_v1.SubscriberClient(credentials=credentials)
try:
    subscriber.create_subscription(
    name=subscription_name, 
    topic=topic_name,
    ack_deadline_seconds=60,  #This limits the likelihood google will redeliver a recieved message, default is 10s.
    )
    print ('Created Subscriptions in Project \n')


    print ('Listing Subscriptions in Your Project %s : ' % os.getenv('GOOGLE_CLOUD_PROJECT'))
    for subscription in subscriber.list_subscriptions(subscriber.project_path(os.environ['GOOGLE_CLOUD_PROJECT'])):
        print('\t', subscription.name)



except:
    e = sys.exc_info()[1]
    print( "Error: %s \n" % e )


Created Subscriptions in Project 

Listing Subscriptions in Your Project prefab-rampart-794 : 
	 projects/prefab-rampart-794/subscriptions/ALEXDEMO_CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310


## Subscription View in Google Cloud Console

Subscriptions are also avaliable for viewing in Google Cloud Console (https://console.cloud.google.com/).  Navigate to Cloud Pub/Sub and click Subscription.  If you click your Subscription Name, it will open up the details about that Subscription.  You can see the all queued messages and core settings which are set to default settings as we did not specify special settings and the functions above used the defaults.  

Another thing shown in this view is the total queued messages from GCP in the Subscription.


## Pull a Single Message from CME 

The following will do a simple message pull from your Subscription and print it out locally.  There are extensive examples on data pulling from a Subscription including batch and async (https://cloud.google.com/pubsub/docs/pull).  



In [17]:
#Pull 1 Message
print ('Pulling a Single Message and Displaying:')

CME_DATA = subscriber.pull(subscription_name, max_messages=1)

#Print that Message
print (CME_DATA)


Pulling a Single Message and Displaying:
received_messages {
  ack_id: "TgQhIT4wPkVTRFAGFixdRkhRNxkIaFEOT14jPzUgKEURAQgUBXx9cUFPdVVeGgdRDRlyfGcjbAkXUgRGVnlVWRENem1cVzhYDxl7e2F2bl4VAwpHUn13wszCqPLBIR1tNY2h8KFASony3-N2Zho9XxJLLD5-PT1FQV5AEkw2CERJUytDCypYEQ"
  message {
    data: ":|\027\000\353\336\260\266\260n\346\025@\000\013\000/\000\001\000\t\000\257\266\220\266\260n\346\025\204\000\000(\000\001\331\376\304\022\017\000\000\000\211E\303$\000\000\000\000\000j\317\245>\'\001\000\003\000\000\000K)\000\000\0001\000\000\000\000\000\000"
    attributes {
      key: "Channel"
      value: "310"
    }
    attributes {
      key: "MsgSeqNum"
      value: "1539130"
    }
    attributes {
      key: "SendingTime"
      value: "1578070424712"
    }
    message_id: "920960344628213"
    publish_time {
      seconds: 1578070424
      nanos: 736000000
    }
  }
}



# Delete Subscriptions

You can also use the Python SDK to delete your Cloud Pub/Sub Subscriptions.  The following will attempt to delete ALL the subscriptions in your Project.   


In [18]:
#List Subscriptions in My Project / Delete Subscription
delete = True

subscriber = pubsub_v1.SubscriberClient()
project_path = subscriber.project_path(os.environ['GOOGLE_CLOUD_PROJECT'])

if not delete: 

    print ('Did you mean to Delete all Subscriptions?  If yes, then set delete = True')

for subscription in subscriber.list_subscriptions(project_path):    
    #Delete Subscriptions
    if delete: 
        subscriber.delete_subscription(subscription.name)
        print ("\tDeleted: {}".format(subscription.name))
    else:
        print("\tActive Subscription: {}".format(subscription.name))

/anaconda3/envs/python3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


	Deleted: projects/prefab-rampart-794/subscriptions/ALEXDEMO_CERT.SSCL.GCP.MD.RT.CMEG.FIXBIN.v01000.INCR.310


# Summary

This notebook went through the bare minimum needed to create a Cloud Pub/Sub Subscription against the CME Smart Stream on GCP solutions.  





#  Questions?

If you have questions or think we can update this to additional use cases, please use the Issues feature in Github or reach out to CME Sales team at markettechsales@cmegroup.com


